**First clone the github repository**

In [ ]:
%cd /content
! git clone https://github.com/Tikquuss/lwd
%cd lwd/scripts

In [ ]:
main_path = "/content"

# **Set-up** 
*Once this section is configured, you can comment (optionally) the previous cell and run everything at once (Runtime -> Run all).*  
**Once everything is executed, a csv file will be automatically created containing all the losses on the test data for each model.**

Uncomment the line corresponding to your function, and choose the value xxx accordingly:
* For Black&Scholes, INPUT_DIM must remain at 1.
* For Gaussian basket options, INPUT_DIM can take any integer value greater than or equal to 1.


In [ ]:
f_name = "Black&Scholes"
#f_name = "Gaussian basket options" # vary INPUT_DIM : 1,...,7,..., 20,...

INPUT_DIM = 1

In [ ]:
try:
    %tensorflow_version 1.x
    %matplotlib inline
except Exception:
    pass

import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

import random

from utils import plot_stat, MLP, Siren, train, test, global_stat, reshape, to_csv
from twin_net_tf import graph, get_diffML_data_loader, BlackScholes, Bachelier, test as twin_net_tf_test 
from twin_net_tf_siren import test as twin_net_tf_test_siren

In [ ]:
# Global
max_epoch = 10000
batch_sizes = [16, 64, 512, 1024, 1024] # batch_size
nTrains = [20, 100, 1000, 10000, 100000] # number of training examples
nTests = [10000]*5 # number of test examples

train_seed, test_seed = 0, 1 # for reproducibility
learning_rate = 3e-5
improving_limit = float("inf") # no limit
HIDDEN_DIM = 20
N_HIDDEN = 4
OUTPUT_DIM = 1

# MLP
activation_function = F.softplus
deriv_activation_function = torch.sigmoid # for twin_net_pytorch
mlp_model_kwargs = {"in_features" : INPUT_DIM, # depends on the function
                    "hidden_features" : HIDDEN_DIM, 
                    "hidden_layers" : N_HIDDEN, 
                    "out_features": OUTPUT_DIM, 
                    "activation_function" : activation_function, 
                    "deriv_activation_function" : deriv_activation_function,
                   }

generator_kwargs = {"hidden_units" : HIDDEN_DIM, 
                    "hidden_layers" : N_HIDDEN}

# hyperparameters in the different loss functions to express a tradeoff between y loss and dy loss
# Leave None and None instead of 1 and 1, this will be managed automatically.
loss_config = {'alpha': None, "beta" : None} 

# twin_net
learning_rate_schedule = [(0.0, 1.0e-8), (0.2, 0.1), (0.6, 0.01), (0.9, 1.0e-6), (1.0, 1.0e-8)]

# Siren
first_omega_0 = 30.
hidden_omega_0 = 30.
outermost_linear = True

siren_model_kwargs = {"in_features" : INPUT_DIM, 
                      "hidden_features" : HIDDEN_DIM, 
                      "hidden_layers" : N_HIDDEN, 
                      "out_features": OUTPUT_DIM, 
                      "outermost_linear" : outermost_linear, 
                      "first_omega_0" : first_omega_0, 
                      "hidden_omega_0" : hidden_omega_0}

# **To avoid repeating the same code too much**

In [ ]:
if f_name == "Black&Scholes" :
    generator = BlackScholes()
    # for twin_net_tf
    graph_name = "Black & Scholes"
elif f_name == "Gaussian basket options" : 
    generator = Bachelier(n = INPUT_DIM)
    # for twin_net_tf
    graph_name = "Bachelier dimension %d" % INPUT_DIM
    
csv_path = os.path.join(main_path, f_name + ".csv")
import os
if not os.path.exists(main_path):
    os.makedirs(main_path)

stats_dic = {}
tests_loss = {}

In [ ]:
def run_train(name, generator, with_derivative, model_class, model_kwargs, normalize, learning_rate_schedule = None):
    
    global nTrains, nTests, batch_sizes, train_seed, test_seed, learning_rate
    global max_epoch, improving_limit
    global loss_config 

    model_list, loss_list, stat_list = [], [], []
    
    for nTrain, nTest, batch_size in zip(nTrains, nTests, batch_sizes) : 

        train_dataloader, test_dataloader, xAxis, vegas, config = get_diffML_data_loader(
            generator = generator, 
            nTrain = nTrain, nTest = nTest, 
            train_seed = train_seed, test_seed = test_seed, 
            batch_size = batch_size, 
            with_derivative = with_derivative,
            normalize = normalize
        )
        
        config["learning_rate_schedule"] = learning_rate_schedule
        config.update({key : value for key, value in loss_config.items() if value})
        
        model = model_class(**model_kwargs)
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

        model, stats, best_loss = train(name, model, train_dataloader, optimizer, criterion, config, 
                                        with_derivative, max_epoch = max_epoch, improving_limit = improving_limit)
        
        plot_stat(stats, with_derivative = with_derivative)

        (test_loss, r_y, r_dydx), (x_list, y_list, dydx_list, y_pred_list, dydx_pred_list) = test(
            name, model, test_dataloader, criterion, config, with_derivative
        )
        
        xy = [(x[0], y[0]) for x, y in zip(x_list, y_list)]
        xy_pred = [(x[0], y[0]) for x, y in zip(x_list, y_pred_list)]
      
        if with_derivative :
            xdydx = [(x[0], y[0]) for x, y in zip(x_list, dydx_list)]
            xdydx_pred = [(x[0], y) for x, y in zip(x_list, dydx_pred_list)]

            fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize = (15,3))
        else :
            fig, ax1 = plt.subplots(1, 1, sharex=True, figsize = (15,3))

        fig.suptitle('')
        
        ax1.scatter(*zip(*xy), label = "y")
        ax1.scatter(*zip(*xy_pred), label = "ypred")
        ax1.set(xlabel='x', ylabel='y, y_pred')
        ax1.legend()
        if with_derivative :
            ax2.scatter(*zip(*xdydx), label = "dy")
            ax2.scatter(*zip(*xdydx_pred), label = "dy pred")
            ax2.set(xlabel='x', ylabel='dy, dy_pred')
            ax2.legend()

        model_list.append(model)
        loss_list.append((test_loss, r_y, r_dydx))
        stat_list.append(stats)
        
    return model_list, loss_list, stat_list

def run_diffML_train(name, generator, generator_kwargs, show_graph_per_axis = False, input_dim = None, 
                     siren = False, normalize = True):
    global nTrains, nTests, batch_sizes, train_seed, test_seed
    global max_epoch
    global first_omega_0, hidden_omega_0, outermost_linear

    regressor_list, loss_list = [], []
    ass = {}
    ass["normal"] = ass["differential"] = []

    for nTrain, nTest, batch_size in zip(nTrains, nTests, batch_sizes) :
    
        if siren :
            dic_loss, regressor, dtrain, dtest, dydxTest, values, deltas, xAxis, vegas = twin_net_tf_test_siren(
                  generator, [nTrain], 
                  nTrain, nTest, 
                  trainSeed = train_seed, testSeed = test_seed, weightSeed = 0, 
                  deltidx=0,
                  generator_kwargs = generator_kwargs,
                  epochs = max_epoch,
                  first_omega_0 = first_omega_0, 
                  hidden_omega_0 = hidden_omega_0, 
                  outermost_linear = outermost_linear,
                  normalize = normalize,
                  improving_limit = improving_limit,
                  min_batch_size = batch_size
                  ) 
        else :
            dic_loss, regressor, dtrain, dtest, dydxTest, values, deltas, xAxis, vegas = twin_net_tf_test(
                  generator, [nTrain], 
                  nTrain, nTest, 
                  trainSeed = train_seed, testSeed = test_seed, weightSeed = 0, 
                  deltidx=0,
                  generator_kwargs = generator_kwargs,
                  normalize = normalize,
                  epochs = max_epoch,
                  min_batch_size = batch_size
                  )
        
        plot_stat(regressor.stats["normal"], with_derivative = with_derivative)
        plot_stat(regressor.stats["differential"], with_derivative = with_derivative)

        yTest = dtest[1]
        sizes = [nTrain]
        # show predicitions
        graph(name, values, xAxis, "", "values", yTest, [nTrain], True)
        # show deltas
        graph(name, deltas, xAxis, "", "deltas", dydxTest, [nTrain], True)

        if show_graph_per_axis :
            assert input_dim
            for i in range(input_dim) :
                xAxis  = np.array([[x[i]] for x in dtest[0]])
                # show predicitions
                graph("%s x%d vs y" % (name, (i+1)), values, xAxis, "", "values", yTest, [nTrain], True)
                # show deltas
                graph("%s x%d vs dxdy" % (name, (i+1)), deltas, xAxis, "", "deltas", dydxTest, [nTrain], True)

        a = dic_loss['standard_loss']["yloss"][-1]
        b = dic_loss['standard_loss']["dyloss"][-1]
        normal = (a+b, a, b)
        a = dic_loss['differential_loss']["yloss"][-1]
        b = dic_loss['differential_loss']["dyloss"][-1]
        differential = (a+b, a, b)
        loss = {"normal" : normal, "differential" : differential}

        ass["normal"].append(normal)
        ass["differential"].append(differential)
        regressor_list.append(regressor) 
        
    return regressor_list, ass

# **1) Normal Training**

In [ ]:
name = "net"
with_derivative = False
key1 = "normal_training"
stats_dic[key1] = {}
tests_loss[key1] = {}

## **1.1) with MLP**

In [ ]:
model_class = MLP
key2 = "mlp"
stats_dic[key1][key2] = {}
tests_loss[key1][key2] = {}

### **1.1.1) normalize = False**

In [ ]:
key3 = "no_normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = False, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = False, 
    learning_rate_schedule = learning_rate_schedule)

### **1.1.2) normalize = True**

In [ ]:
key3 = "normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = True, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = True, 
    learning_rate_schedule = learning_rate_schedule)

## **1.2) with Siren**

In [ ]:
model_class = Siren
key2 = "siren"
stats_dic[key1][key2] = {}
tests_loss[key1][key2] = {}


### **1.2.1) normalize = False**

In [ ]:
key3 = "no_normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = False, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = False, 
    learning_rate_schedule = learning_rate_schedule)

### **1.2.2) normalize = True**

In [ ]:
key3 = "normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = True, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = True, 
    learning_rate_schedule = learning_rate_schedule)

# **2) Sobolev Training**

In [ ]:
name = "net"
with_derivative = True
key1 = "sobolev_training"
stats_dic[key1] = {}
tests_loss[key1] = {}

## **2.1) with MLP**

In [ ]:
model_class = MLP
key2 = "mlp"
stats_dic[key1][key2] = {}
tests_loss[key1][key2] = {}

### **2.1.1) normalize = False**

In [ ]:
key3 = "no_normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = False, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = False, 
    learning_rate_schedule = learning_rate_schedule)

### **2.1.2) normalize = True**

In [ ]:
key3 = "normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = True, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = True, 
    learning_rate_schedule = learning_rate_schedule)

## **2.2) with Siren**

In [ ]:
model_class = Siren
key2 = "siren"
stats_dic[key1][key2] = {}
tests_loss[key1][key2] = {}

### **2.2.1) normalize = False**

In [ ]:
key3 = "no_normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = False, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = False, 
    learning_rate_schedule = learning_rate_schedule)

### **2.2.2) normalize = True**

In [ ]:
key3 = "normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = True, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = True, 
    learning_rate_schedule = learning_rate_schedule)

# **3) twin_net tensorflow**

In [ ]:
key1 = "twin_net_tf"
key1_1 = "%s_normal" % key1
key1_2 = "%s_differential" % key1
stats_dic[key1_1] = {}
stats_dic[key1_2] = {}
tests_loss[key1_1] = {}
tests_loss[key1_2] = {}

## **3.1) with MLP**

In [ ]:
key2 = "mlp"
stats_dic[key1_1][key2] = {}
stats_dic[key1_2][key2] = {}
tests_loss[key1_1][key2] = {}
tests_loss[key1_2][key2] = {}

### **3.1.1) normalize = False**

In [ ]:
key3 = "no_normalize"
stats_dic[key1_1][key2][key3] = [None, None]
stats_dic[key1_2][key2][key3] = [None, None]
tests_loss[key1_1][key2][key3] = [None, None]
tests_loss[key1_2][key2][key3] = [None, None]

regressor, loss = run_diffML_train(
              name = graph_name, 
              generator = generator, 
              generator_kwargs = generator_kwargs ,
              show_graph_per_axis = True, 
              input_dim = INPUT_DIM,
              normalize = False
              )
stats_dic[key1_1][key2][key3][0] = stats_dic[key1_1][key2][key3][1] = [r.stats["normal"] for r in regressor]
stats_dic[key1_2][key2][key3][0] = stats_dic[key1_2][key2][key3][1] = [r.stats["differential"] for r in regressor]
tests_loss[key1_1][key2][key3][0] = tests_loss[key1_1][key2][key3][1] = loss["normal"]
tests_loss[key1_2][key2][key3][0] = tests_loss[key1_2][key2][key3][1] = loss["differential"]

### **3.1.2) normalize = True**

In [ ]:
key3 = "normalize"
stats_dic[key1_1][key2][key3] = [None, None]
stats_dic[key1_2][key2][key3] = [None, None]
tests_loss[key1_1][key2][key3] = [None, None]
tests_loss[key1_2][key2][key3] = [None, None]

regressor, loss = run_diffML_train(
              name = graph_name, 
              generator = generator, 
              generator_kwargs = generator_kwargs ,
              show_graph_per_axis = True, 
              input_dim = INPUT_DIM,
              normalize = True
              )

stats_dic[key1_1][key2][key3][0] = stats_dic[key1_1][key2][key3][1] = [r.stats["normal"] for r in regressor]
stats_dic[key1_2][key2][key3][0] = stats_dic[key1_2][key2][key3][1] = [r.stats["differential"] for r in regressor]
tests_loss[key1_1][key2][key3][0] = tests_loss[key1_1][key2][key3][1] = loss["normal"]
tests_loss[key1_2][key2][key3][0] = tests_loss[key1_2][key2][key3][1] = loss["differential"]

## **3.2) with Siren**

In [ ]:
key2 = "siren"
stats_dic[key1_1][key2] = {}
stats_dic[key1_2][key2] = {}
tests_loss[key1_1][key2] = {}
tests_loss[key1_2][key2] = {}

### **3.2.1) normalize = False**

In [ ]:
key3 = "no_normalize"
stats_dic[key1_1][key2][key3] = [None, None]
stats_dic[key1_2][key2][key3] = [None, None]
tests_loss[key1_1][key2][key3] = [None, None]
tests_loss[key1_2][key2][key3] = [None, None]

regressor, loss = run_diffML_train(
              name = graph_name, 
              generator = generator, 
              generator_kwargs = generator_kwargs ,
              show_graph_per_axis = True, 
              input_dim = INPUT_DIM,
              siren = True, # Siren
              normalize = False
              )

stats_dic[key1_1][key2][key3][0] = stats_dic[key1_1][key2][key3][1] = [r.stats["normal"] for r in regressor]
stats_dic[key1_2][key2][key3][0] = stats_dic[key1_2][key2][key3][1] = [r.stats["differential"] for r in regressor]
tests_loss[key1_1][key2][key3][0] = tests_loss[key1_1][key2][key3][1] = loss["normal"]
tests_loss[key1_2][key2][key3][0] = tests_loss[key1_2][key2][key3][1] = loss["differential"]

### **3.2.2) normalize = True**

In [ ]:
key3 = "normalize"
stats_dic[key1_1][key2][key3] = [None, None]
stats_dic[key1_2][key2][key3] = [None, None]
tests_loss[key1_1][key2][key3] = [None, None]
tests_loss[key1_2][key2][key3] = [None, None]

regressor, loss = run_diffML_train(
              name = graph_name, 
              generator = generator, 
              generator_kwargs = generator_kwargs ,
              show_graph_per_axis = True, 
              input_dim = INPUT_DIM,
              siren = True, # Siren
              normalize = True
              )

stats_dic[key1_1][key2][key3][0] = stats_dic[key1_1][key2][key3][1] = [r.stats["normal"] for r in regressor]
stats_dic[key1_2][key2][key3][0] = stats_dic[key1_2][key2][key3][1] = [r.stats["differential"] for r in regressor]
tests_loss[key1_1][key2][key3][0] = tests_loss[key1_1][key2][key3][1] = loss["normal"]
tests_loss[key1_2][key2][key3][0] = tests_loss[key1_2][key2][key3][1] = loss["differential"]

# **4) twin_net pytorch**

In [ ]:
name = "twin_net"
with_derivative = True
key1 = "twin_net_pytorch"
stats_dic[key1] = {}
tests_loss[key1] = {}

## **4.1) with MLP**

In [ ]:
model_class = MLP
key2 = "mlp"
stats_dic[key1][key2] = {}
tests_loss[key1][key2] = {}

### **4.1.1) normalize = False**

In [ ]:
key3 = "no_normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = False, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = False, 
    learning_rate_schedule = learning_rate_schedule)

### **4.1.2) normalize = True**

In [ ]:
key3 = "normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = True, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = mlp_model_kwargs,
    normalize = True, 
    learning_rate_schedule = learning_rate_schedule)

## **4.2) with Siren**

In [ ]:
model_class = Siren
key2 = "siren"
stats_dic[key1][key2] = {}
tests_loss[key1][key2] = {}

### **4.2.1) normalize = False**

In [ ]:
key3 = "no_normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = False, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = False, 
    learning_rate_schedule = learning_rate_schedule)

### **4.2.2) normalize = True**

In [ ]:
key3 = "normalize"
stats_dic[key1][key2][key3] = [None, None]
tests_loss[key1][key2][key3] = [None, None]

In [ ]:
model, tests_loss[key1][key2][key3][0], stats_dic[key1][key2][key3][0] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = True, 
    learning_rate_schedule = None)

In [ ]:
model, tests_loss[key1][key2][key3][1], stats_dic[key1][key2][key3][1] = run_train(
    name = name, 
    generator = generator, 
    with_derivative = with_derivative, 
    model_class = model_class, 
    model_kwargs = siren_model_kwargs,
    normalize = True, 
    learning_rate_schedule = learning_rate_schedule)

# **5) Global Stats**

In [ ]:
import pickle, os

for nTrain, sd in reshape(dic = stats_dic, nTrains = nTrains).items() :
    
    print("nTrain %d" % nTrain)

    global_stat(stats_dic = sd, suptitle = graph_name)

    file_path = os.path.join(main_path, str(nTrain) + ".pkl")
    pickle.dump(sd, open(file_path, 'wb'))
    #stats_dic = pickle.load(open(file_path, 'rb'))

In [ ]:
for nTrain, tl in reshape(dic = tests_loss, nTrains = nTrains).items() :
    print("nTrain %d" % nTrain)
    rows, result = to_csv(dico = tl, csv_path = csv_path, n_samples = str(nTrain), mode='a+')